In [1]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=8a1ce3aa04f56e8316db14f838e69bee2dbab42c1b4a791a97a991e2f1a35e00
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
def text_encoding(text):
  return(model.encode(text))

In [3]:
import numpy as np
import pandas as pd

In [4]:
df_train_prompts = pd.read_csv('prompts_train.csv')
df_train_summaries = pd.read_csv('summaries_train.csv')
df_test_prompts = pd.read_csv('prompts_test.csv')
df_test_summaries = pd.read_csv('summaries_test.csv')

In [5]:
df_train = df_train_summaries.merge(df_train_prompts, on='prompt_id')
df_test = df_test_summaries.merge(df_test_prompts, on='prompt_id')

# Encoding the text

In [6]:
model = SentenceTransformer('all-mpnet-base-v2')


In [7]:
df_train["text_encoded"] = df_train['text'].apply(text_encoding)
df_test["text_encoded"] = df_test['text'].apply(text_encoding)

# Fitting the model

In [8]:
from sklearn import svm
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor


rf = RandomForestRegressor(random_state = 42)
decision_tree_regressor = tree.DecisionTreeRegressor()
decision_tree_regressor.fit(df_train["text_encoded"].to_list(),df_train[["content","wording"]])

DecisionTreeRegressor()

# Predict

In [15]:
df_train[["wording_predicted","content_predicted"]] = decision_tree_regressor.predict(df_train["text_encoded"].to_list())

# Evaluate prediction

In [16]:
from sklearn.metrics import r2_score

# Calculate R2 score for each model
r2_content = r2_score(df_train['content'], df_train["content_predicted"])
r2_wording = r2_score(df_train['wording'], df_train["wording_predicted"])

print(f"Content Model R2 Score: {r2_content}")
print(f"Wording Model R2 Score: {r2_wording}")

Content Model R2 Score: 0.5041835455432853
Wording Model R2 Score: 0.49687159427327887


In [17]:
import numpy as np

def compute_mcrmse():
    def rmse(actual, predictions):
        return np.sqrt(np.mean((actual - predictions)**2))

    rmse_content = rmse(df_train['content'], df_train["content_predicted"])
    rmse_wording = rmse(df_train['wording'], df_train["content_predicted"])

    mcrmse = np.mean([rmse_content, rmse_wording])

    return mcrmse, rmse_content, rmse_wording

In [18]:
print("mcrmse")
print(compute_mcrmse())

mcrmse
(0.3761732127865448, 0.7347701214500121, 0.017576304123077518)


In [22]:
df_test[["wording_predicted","content_predicted"]] = decision_tree_regressor.predict(df_test["text_encoded"].to_list())

In [24]:
df_test.head()

,student_id,prompt_id,text,prompt_question,prompt_title,prompt_text,text_encoded,wording_predicted,content_predicted
0,000000ffffff,abc123,Example text 1,Summarize...,Example Title 1,Heading\nText...,"[0.032378275, -0.047635112, -0.012262541, 0.00...",-1.547163,-1.461245
1,222222cccccc,abc123,Example text 3,Summarize...,Example Title 1,Heading\nText...,"[0.06417478, -0.06364345, -0.011845122, 0.0119...",-1.355562,-0.955801
2,111111eeeeee,def789,Example text 2,Summarize...,Example Title 2,Heading\nText...,"[0.04659741, -0.03757003, -0.01091581, 0.00152...",-1.355562,-0.955801
3,333333dddddd,def789,Example text 4,Summarize...,Example Title 2,Heading\nText...,"[0.04591531, -0.041503184, -0.011434707, 0.019...",-1.355562,-0.955801


In [25]:
submission_df = df_test[["student_id","wording_predicted","content_predicted"]]

In [26]:
submission_df.to_csv('submission.csv')